# Disaster Image Classification

Made By:
- Austin Kane - 27022229232
- Andreas Immanuel Lukito - 2702211595

In [7]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2, EfficientNetB0, EfficientNetB2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow_addons.metrics import F1Score
from PIL import ImageFile

## Set the Parameters

In [8]:
IMAGE_SIZE = (224, 224) # MobileNetV2's input size is 224x224 
BATCH_SIZE = 32 
NUM_CLASSES = 4 # 4 classes of data
DATA_DIR = 'Data'

In [ ]:
datagen = ImageDataGenerator(
    rescale=1./255, # Normalize the images
    validation_split=0.2, # Train Val Split
    # Image Augmentation
    horizontal_flip=True,
    vertical_flip=True,
    rotation_range=20,
    zoom_range=0.2
)

In [ ]:
train_generator = datagen.flow_from_directory(
    DATA_DIR,
    target_size=IMAGE_SIZE, # Scale the image to fit model's input
    batch_size=BATCH_SIZE, 
    class_mode='categorical',
    subset='training',
    shuffle=True
)

Found 2754 images belonging to 4 classes.


In [ ]:
val_generator = datagen.flow_from_directory(
    DATA_DIR,
    target_size=IMAGE_SIZE, # Scale the image to fit model's input
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation',
    shuffle=False
)

Found 686 images belonging to 4 classes.


## MobileNetV2

In [12]:
base_model = MobileNetV2(
    weights='imagenet',
    include_top=False,
    input_shape=(*IMAGE_SIZE, 3)
)
base_model.trainable = False

In [13]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.3)(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.3)(x)
predictions = Dense(NUM_CLASSES, activation='softmax')(x)

MobileNet = Model(inputs=base_model.input, outputs=predictions)

In [ ]:
f1_metric = F1Score(num_classes=NUM_CLASSES, average='macro')
MobileNet.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=[f1_metric])

In [16]:
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [ ]:
history = MobileNet.fit(
    train_generator,
    epochs=25,
    validation_data=val_generator
)

Epoch 1/10
87/87 [==============================] - 86s 984ms/step - loss: 0.9726 - f1_score: 0.5642 - val_loss: 0.6394 - val_f1_score: 0.7276
Epoch 2/10
87/87 [==============================] - 83s 948ms/step - loss: 0.7064 - f1_score: 0.6871 - val_loss: 0.5855 - val_f1_score: 0.7517
Epoch 3/10
87/87 [==============================] - 78s 897ms/step - loss: 0.6452 - f1_score: 0.7198 - val_loss: 0.5953 - val_f1_score: 0.7396
Epoch 4/10
87/87 [==============================] - 76s 877ms/step - loss: 0.6227 - f1_score: 0.7296 - val_loss: 0.5254 - val_f1_score: 0.7597
Epoch 5/10
87/87 [==============================] - 78s 891ms/step - loss: 0.5965 - f1_score: 0.7403 - val_loss: 0.5415 - val_f1_score: 0.7420
Epoch 6/10
87/87 [==============================] - 73s 844ms/step - loss: 0.6078 - f1_score: 0.7496 - val_loss: 0.5590 - val_f1_score: 0.7523
Epoch 7/10
87/87 [==============================] - 72s 823ms/step - loss: 0.5763 - f1_score: 0.7452 - val_loss: 0.5312 - val_f1_score: 0.7783

In [27]:
loss, f1 = MobileNet.evaluate(val_generator)
print(f"Validation F1-Score: {f1:.4f}")

22/22 [==============================] - 11s 486ms/step - loss: 0.4827 - f1_score: 0.8074
Validation F1-Score: 0.8074


## EfficientNetB0

In [20]:
base_model = EfficientNetB0(
    weights='imagenet',
    include_top=False,
    input_shape=(*IMAGE_SIZE, 3)
)
base_model.trainable = False

In [21]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.3)(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.3)(x)
predictions = Dense(NUM_CLASSES, activation='softmax')(x)

EffNetB0 = Model(inputs=base_model.input, outputs=predictions)

In [22]:
EffNetB0.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=[f1_metric])

In [23]:
history = EffNetB0.fit(
    train_generator,
    epochs=10,
    validation_data=val_generator
)

Epoch 1/10
87/87 [==============================] - 93s 1s/step - loss: 1.2691 - f1_score: 0.3491 - val_loss: 1.2425 - val_f1_score: 0.1546
Epoch 2/10
87/87 [==============================] - 90s 1s/step - loss: 1.2468 - f1_score: 0.1583 - val_loss: 1.2424 - val_f1_score: 0.1546
Epoch 3/10
87/87 [==============================] - 89s 1s/step - loss: 1.2508 - f1_score: 0.1590 - val_loss: 1.2450 - val_f1_score: 0.1546
Epoch 4/10
87/87 [==============================] - 91s 1s/step - loss: 1.2528 - f1_score: 0.1554 - val_loss: 1.2475 - val_f1_score: 0.1546
Epoch 5/10
87/87 [==============================] - 91s 1s/step - loss: 1.2522 - f1_score: 0.1568 - val_loss: 1.2422 - val_f1_score: 0.1546
Epoch 6/10
87/87 [==============================] - 71s 813ms/step - loss: 1.2539 - f1_score: 0.1559 - val_loss: 1.2447 - val_f1_score: 0.1546
Epoch 7/10
87/87 [==============================] - 73s 843ms/step - loss: 1.2548 - f1_score: 0.1543 - val_loss: 1.2446 - val_f1_score: 0.1546
Epoch 8/10
87/

In [ ]:
loss, acc = EffNetB0.evaluate(val_generator)
print(f"Validation F1-Score: {acc:.4f}")    

22/22 [==============================] - 19s 850ms/step - loss: 1.2427 - f1_score: 0.1546
Validation Accuracy: 0.1546


## EfficientNetB2

In [2]:
IMAGE_SIZE = (260, 260) # EfficientNetB2's input size is 224x224 
BATCH_SIZE = 32 
NUM_CLASSES = 4 # 4 classes of data
DATA_DIR = 'Data'

In [3]:
datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    horizontal_flip=True,
    rotation_range=20,
    zoom_range=0.2
)

In [4]:
train_generator = datagen.flow_from_directory(
    DATA_DIR,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training',
    shuffle=True
)

Found 2754 images belonging to 4 classes.


In [5]:
val_generator = datagen.flow_from_directory(
    DATA_DIR,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation',
    shuffle=False
)

Found 686 images belonging to 4 classes.


In [42]:
base_model = EfficientNetB2(
    weights='imagenet',
    include_top=False,
    input_shape=(*IMAGE_SIZE, 3)
)
base_model.trainable = False

31793152/31790344 [==============================] - 3s 0us/step


In [43]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.3)(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.3)(x)
predictions = Dense(NUM_CLASSES, activation='softmax')(x)

EffNetB2 = Model(inputs=base_model.input, outputs=predictions)

In [44]:
EffNetB2.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [45]:
history = EffNetB2.fit(
    train_generator,
    epochs=10,
    validation_data=val_generator
)

Epoch 1/10
87/87 [==============================] - 97s 1s/step - loss: 1.2705 - accuracy: 0.4230 - val_loss: 1.2530 - val_accuracy: 0.4475
Epoch 2/10
87/87 [==============================] - 92s 1s/step - loss: 1.2543 - accuracy: 0.4423 - val_loss: 1.2438 - val_accuracy: 0.4475
Epoch 3/10
87/87 [==============================] - 89s 1s/step - loss: 1.2570 - accuracy: 0.4383 - val_loss: 1.2435 - val_accuracy: 0.4475
Epoch 4/10
87/87 [==============================] - 91s 1s/step - loss: 1.2575 - accuracy: 0.4466 - val_loss: 1.2442 - val_accuracy: 0.4475
Epoch 5/10
87/87 [==============================] - 92s 1s/step - loss: 1.2515 - accuracy: 0.4466 - val_loss: 1.2443 - val_accuracy: 0.4475
Epoch 6/10
87/87 [==============================] - 89s 1s/step - loss: 1.2541 - accuracy: 0.4466 - val_loss: 1.2453 - val_accuracy: 0.4475
Epoch 7/10
87/87 [==============================] - 89s 1s/step - loss: 1.2534 - accuracy: 0.4466 - val_loss: 1.2506 - val_accuracy: 0.4475
Epoch 8/10
87/87 [==

In [46]:
loss, acc = EffNetB2.evaluate(val_generator)
print(f"Validation Accuracy: {acc:.4f}")

22/22 [==============================] - 18s 797ms/step - loss: 1.2423 - accuracy: 0.4475
Validation Accuracy: 0.4475
